In [ ]:
import sys
!{sys.executable} -m pip install deepchemimport deepchem
!{sys.executable} -m pip install PyTDC
!{sys.executable} -m pip install transformers
!{sys.executable} -m pip install xgboost
!{sys.executable} -m pip install huggingface-hub
!{sys.executable} -m pip install tensorflow
!{sys.executable} -m wandb login 6d4f2c82247727cc1e4c0fc1fd8eebf7f7d85ee7


In [1]:
import numpy as np
import torch
import deepchem as dc
import logging
from tokenizer import SmilesTokenizer, TokenizerStatus, UNK_TOKEN
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
import xgboost as xgb
from tqdm import tqdm
from deepchem.models import GraphConvModel
from sklearn.tree import DecisionTreeClassifier
from simpletransformers.classification import ClassificationModel, ClassificationArgs
from sklearn.feature_extraction.text import CountVectorizer

logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)



from tdc.single_pred import ADME
data = ADME(name = 'CYP2D6_Veith')
split = data.get_split()



c:\Users\Hurle\AppData\Local\Programs\Python\Python39\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Skipped loading modules with pytorch-geometric dependency, missing a dependency. No module named 'torch_geometric'
Skipped loading modules with pytorch-geometric dependency, missing a dependency. cannot import name 'DMPNN' from 'deepchem.models.torch_models' (c:\Users\Hurle\AppData\Local\Programs\Python\Python39\lib\site-packages\deepchem\models\torch_models\__init__.py)
Skipped loading modules with pytorch-lightning dependency, missing a dependency. No module named 'pytorch_lightning'
Skipped loading some Jax models, missing a dependency. No module named 'jax'
Found local copy...
Loading...
Done!


In [6]:
#TRANSFORMER MODEL
# model = ClassificationModel('roberta', 'seyonec/ChemBERTa-zinc-base-v1', args={'num_train_epochs': 1, 'auto_weights': True},use_cuda = False) # You can set class weights by using the optional weight argument


# model.train_model(split["train"][['Drug','Y']], num_labels=2,args={'wandb_project': 'project-name'})
model = ClassificationModel('roberta', 'outputs/', num_labels=2, use_cuda=False, args={'wandb_project': 'project-name','num_train_epochs': 3})
result, model_outputs, wrong_predictions = model.eval_model(split["valid"][['Drug','Y']], acc=metrics.accuracy_score)

INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.
  0%|          | 3/1313 [00:22<2:43:57,  7.51s/it]
INFO:simpletransformers.classification.classification_utils: Saving features into cached file cache_dir/cached_dev_roberta_128_2_2
Running Evaluation: 100%|██████████| 165/165 [05:54<00:00,  2.15s/it]
ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
INFO:simpletransformers.classification.classification_model: Initializing WandB run for evaluation.
wandb: Currently logged in as: hurleyoe. Use `wandb login --relogin` to force relogin


wandb: WARNING wandb.plots.* functions are deprecated and will be removed in a future release. Please use wandb.plot.* instead.
INFO:simpletransformers.classification.classification_model:{'mcc': 0.4531885472231916, 'tp': 93, 'tn': 1024, 'fp': 29, 'fn': 167, 'auroc': 0.8305829498137189, 'auprc': 0.6172836590496875, 'acc': 0.8507235338918507, 'eval_loss': 0.3884291167624972}


In [20]:
transformer_predictions,_ = model.predict(list(split['valid'].Drug))
print("Transformer Model:")
print("AUROC:", metrics.roc_auc_score(split['valid'].Y,transformer_predictions))
print("Accuracy:", metrics.accuracy_score(split['valid'].Y,transformer_predictions))


INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.
100%|██████████| 165/165 [07:22<00:00,  2.68s/it]

AUROC: 0.6650759734093068
Accuracy: 0.8507235338918507


1313

In [24]:
#DECISION TREES
tokenizer = SmilesTokenizer

def get_vocab_from_smiles(smiles_list):
    """Returns a sorted list of all unique tokens in smiles_list."""
    tokenizer = SmilesTokenizer()

    all_tokens = set()
    for smiles in tqdm(smiles_list):
        tokens_list = tokenizer.tokenize(smiles)
        if not isinstance(tokens_list, TokenizerStatus):
            all_tokens.update(tokens_list)
    return sorted(list(all_tokens))


vocab = get_vocab_from_smiles(data.entity1)
vectorizer = CountVectorizer(vocabulary = vocab, tokenizer = tokenizer().tokenize)
X = vectorizer.fit_transform(split['train'].Drug)


dtmodel = DecisionTreeClassifier(random_state=0)
dtmodel.fit(X,split['train'].Y)
decision_tree_predictions = dtmodel.predict(vectorizer.fit_transform(split['valid'].Drug))
print("Decision Tree:")
print("AUROC:", metrics.roc_auc_score(split['valid'].Y,decision_tree_predictions))
print("Accuracy:", metrics.accuracy_score(split['valid'].Y,decision_tree_predictions))
print('\n')

rfmodel = RandomForestClassifier(random_state=0)
rfmodel.fit(X,split['train'].Y)
random_forest_predictions = rfmodel.predict(vectorizer.fit_transform(split['valid'].Drug))
print("Random Forest:")
print("AUROC:", metrics.roc_auc_score(split['valid'].Y,random_forest_predictions))
print("Accuracy:", metrics.accuracy_score(split['valid'].Y,random_forest_predictions))
print('\n')

xgbmodel = xgb.XGBClassifier(objective="binary:logistic", random_state=0)
xgbmodel.fit(X,split['train'].Y)
xgb_predictions = xgbmodel.predict(vectorizer.fit_transform(split['valid'].Drug))
print("XGB:")
print("AUROC:", metrics.roc_auc_score(split['valid'].Y,xgb_predictions))
print("Accuracy:", metrics.accuracy_score(split['valid'].Y,xgb_predictions))


100%|██████████| 13130/13130 [00:01<00:00, 7322.18it/s]


Decision Tree:
AUROC: 0.6513057929724597
Accuracy: 0.7821782178217822


Random Forest:
AUROC: 0.6474121557454892
Accuracy: 0.8316831683168316


XGB:
AUROC: 0.656054131054131
Accuracy: 0.8362528560548362


In [27]:
#GRAPH
featurizer = dc.feat.ConvMolFeaturizer()
loader = dc.data.InMemoryLoader(tasks=["GraphNN"], featurizer = featurizer)
smiles = list(split["train"].Drug.values)
labels = list(split["train"].Y.values)
dataset = loader.create_dataset(zip(smiles, labels))

# Splitter splits the dataset 
    # In this case it's is an equivalent of train_test_split from sklearn
# splitter = dc.splits.RandomSplitter()
    # frac_test is 0.01 because we only use a train and valid as an example
# train, _, _ = splitter.train_valid_test_split(dataset,
 #                                                     frac_train=1,)


graph_conv = GraphConvModel(1,
                            batch_size=50,
                            mode="classification")

graph_conv.fit(dataset, nb_epoch=10)

metric = metrics.accuracy_score

train_scores = graph_conv.evaluate(dataset, [metric])

print(f"Train Scores: {train_scores}")

INFO:deepchem.data.data_loader:Loading raw samples now.
INFO:deepchem.data.data_loader:shard_size: 8192
INFO:deepchem.feat.base_classes:Featurizing datapoint 0
INFO:deepchem.feat.base_classes:Featurizing datapoint 0
INFO:deepchem.feat.base_classes:Featurizing datapoint 0
INFO:deepchem.feat.base_classes:Featurizing datapoint 0
INFO:deepchem.feat.base_classes:Featurizing datapoint 0
INFO:deepchem.feat.base_classes:Featurizing datapoint 0
INFO:deepchem.feat.base_classes:Featurizing datapoint 0
INFO:deepchem.feat.base_classes:Featurizing datapoint 0
INFO:deepchem.feat.base_classes:Featurizing datapoint 0
INFO:deepchem.feat.base_classes:Featurizing datapoint 0
INFO:deepchem.feat.base_classes:Featurizing datapoint 0
INFO:deepchem.feat.base_classes:Featurizing datapoint 0
INFO:deepchem.feat.base_classes:Featurizing datapoint 0
INFO:deepchem.feat.base_classes:Featurizing datapoint 0
INFO:deepchem.feat.base_classes:Featurizing datapoint 0
INFO:deepchem.feat.base_classes:Featurizing datapoint 0


Train Scores: {'metric-1': 0.8817321292568817}


In [49]:
test_smiles = list(split["valid"].Drug.values)
test_labels = list(split["valid"].Y.values)
test_dataset = loader.create_dataset(zip(test_smiles, test_labels))
test_scores = graph_conv.evaluate(test_dataset, [metrics.accuracy_score,metrics.roc_auc_score])

print(f"Test Scores: {test_scores}")

INFO:deepchem.data.data_loader:Loading raw samples now.
INFO:deepchem.data.data_loader:shard_size: 8192
INFO:deepchem.feat.base_classes:Featurizing datapoint 0
INFO:deepchem.feat.base_classes:Featurizing datapoint 0
INFO:deepchem.feat.base_classes:Featurizing datapoint 0
INFO:deepchem.feat.base_classes:Featurizing datapoint 0
INFO:deepchem.feat.base_classes:Featurizing datapoint 0
INFO:deepchem.feat.base_classes:Featurizing datapoint 0
INFO:deepchem.feat.base_classes:Featurizing datapoint 0
INFO:deepchem.feat.base_classes:Featurizing datapoint 0
INFO:deepchem.feat.base_classes:Featurizing datapoint 0
INFO:deepchem.feat.base_classes:Featurizing datapoint 0
INFO:deepchem.feat.base_classes:Featurizing datapoint 0
INFO:deepchem.feat.base_classes:Featurizing datapoint 0
INFO:deepchem.feat.base_classes:Featurizing datapoint 0
INFO:deepchem.feat.base_classes:Featurizing datapoint 0
INFO:deepchem.feat.base_classes:Featurizing datapoint 0
INFO:deepchem.feat.base_classes:Featurizing datapoint 0


Test Scores: {'metric-1': 0.8568164508758568, 'metric-2': 0.8545437942873839}


In [51]:
print("GNN:")
print("AUROC:", test_scores['metric-2'])
print("Accuracy:", test_scores['metric-1'])


GNN:
AUROC: 0.8545437942873839
Accuracy: 0.8568164508758568


1313